In [ ]:
!unzip '/content/archive.zip'

Archive:  /content/archive.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/archive.zip or
        /content/archive.zip.zip, and cannot find /content/archive.zip.ZIP, period.


Data Loader

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(256), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(256), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    ])

In [ ]:
trainset=torchvision.datasets.ImageFolder(root='/content/Training',transform=transform_train)

In [ ]:
testset=torchvision.datasets.ImageFolder(root='/content/Testing',transform=transform_test)

In [ ]:
batch_size = 50

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True)

In [ ]:
testloader=torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=True)

In [ ]:
def imshow(img, title):
    npimg = img.numpy() / 2 + 0.5
    plt.figure(figsize=(batch_size, 1))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()

In [ ]:
def show_batch_images(dataloader):
    images, labels = next(iter(dataloader))
    img = torchvision.utils.make_grid(images)
    imshow(img, title=[str(x.item()) for x in labels])

In [ ]:
 images, labels = next(iter(trainloader))

In [ ]:
print(images.shape)

In [ ]:
for i in range(batch_size):
    show_batch_images(trainloader)

Model_resnetxt

In [ ]:
from torchvision import models

In [ ]:
import torch

In [ ]:
resnetxt= models.resnext50_32x4d(pretrained=True)

In [ ]:
print(resnetxt)

In [ ]:
for param in resnetxt.parameters():
    param.requires_grad = False

In [ ]:
num_classes=4

In [ ]:
in_features = resnetxt.fc.in_features
resnetxt.fc = torch.nn.Linear(in_features, num_classes)

In [ ]:
resnetxt = resnetxt
loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(resnetxt.parameters(), lr=3e-4)

In [ ]:
import copy

In [ ]:
max_epochs=5

In [ ]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [ ]:
def show_images(images, labels, preds):
    preds=preds.detach()
    plt.figure(figsize=(20,20))
    for i, image in enumerate(images):
        plt.subplot(1,20,i+1, xticks=[], yticks=[])
        image = image.numpy().transpose((1,2,0))
        mean = np.array([0.5])
        std = np.array([0.5])
        image = image * std + mean
        image = np.clip(image, 0., 1.)
        plt.imshow(image)
        print(classes[int(np.argmax(preds[i].numpy()))],end=' ' )
        plt.xlabel(f'{classes[int(labels[i].numpy())]}')
    plt.tight_layout()
    plt.show()

In [ ]:
classes=['GLIOMA',' MENINGIOMA','NORMAL','Pituitary']

In [ ]:
loss_epoch_arr = []
min_loss = 10

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = resnetxt(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(resnetxt.state_dict())
            # print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d, Loss: %0.2f' % (i,loss.item()))
            show_images(inputs[0:5],labels[0:5],outputs[0:5])
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
        
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, max_epochs, 
        evaluation(testloader, resnetxt), evaluation(trainloader, resnetxt)))
    plt.plot(loss_epoch_arr)
    plt.show()

In [ ]:
plt.plot(loss_epoch_arr)
plt.show()

FORMAT :
1. predicted values 
2. image
3. real values

In [ ]:
resnetxt.load_state_dict(best_model)
print(evaluation(trainloader, resnetxt), evaluation(testloader, resnetxt))

In [ ]:
for i,data in enumerate(trainloader,0):
    inputs,labels=data
    outputs = resnetxt(inputs)
    print(i)
    show_images(inputs,labels,outputs)

Test result after training:

In [ ]:
for i,data in enumerate(testloader,0):
    inputs,labels=data
    outputs = resnetxt(inputs)
    if i%2==0:
        print(i)
        show_images(inputs,labels,outputs)

In [ ]:
print('Overall accuracy on test data',evaluation(testloader, resnetxt))